In [1]:
# Install the rapidfuzz package for fuzzy string matching
!pip install rapidfuzz

# Install the deep-translator package for translating text
!pip install deep-translator


In [3]:
# Step 2: Import necessary libraries
import pandas as pd
from deep_translator import GoogleTranslator
from rapidfuzz import process, fuzz

# Step 3: Load the datasets
school_list_A = pd.read_csv('data/school_list_A.tsv', sep='\t')
school_list_B = pd.read_csv('data/school_list_B.tsv', sep='\t')
jillla = pd.read_csv('data/jilla.csv')


In [4]:
# Step 4: Inspect the columns of jillla to find the correct column name
print(jillla.columns)


Index(['पाँचथर'], dtype='object')


In [5]:
# Step 5: Use the actual column name in Devanagari script
nepali_district_column = jillla.columns[0]

# Step 6: Translate district names from Nepali to English
district_map = {}
for nepali_district in jillla[nepali_district_column].unique():
    if pd.notna(nepali_district):
        english_district = GoogleTranslator(source='ne', target='en').translate(nepali_district)
        district_map[nepali_district] = english_district


In [6]:
# Step 7: Map translated district names to school_list_A
school_list_A['district_eng'] = school_list_A['district1'].map(district_map)

# Step 8: Handle NaN values in school_list_A
school_list_A.dropna(subset=['district_eng', 'velthuis'], inplace=True)


In [7]:
# Step 9: Prepare the results DataFrame and match schools
results = []

# Match schools in Source A to Source B within the same district
for _, school_a in school_list_A.iterrows():
    district = school_a['district_eng']
    if pd.isna(district):
        continue  # Skip if district mapping is not found
    
    candidates = school_list_B[school_list_B['district'] == district]
    
    # Combine name and previous names in Source B for fuzzy matching
    names_b = candidates['name'].dropna().tolist()
    if 'old_name1' in candidates.columns:
        names_b += candidates['old_name1'].dropna().tolist()
    if 'old_name2' in candidates.columns:
        names_b += candidates['old_name2'].dropna().tolist()
    if 'old_name3' in candidates.columns:
        names_b += candidates['old_name3'].dropna().tolist()
    
    # Perform fuzzy matching
    matches = process.extract(school_a['velthuis'], names_b, scorer=fuzz.ratio, limit=1)
    
    if matches:
        best_match = matches[0]
        # Find the matching school_id from candidates
        matched_school = candidates[candidates.isin([best_match[0]]).any(axis=1)].iloc[0]
        results.append({
            'school_id_a': school_a['school_id'],
            'school_id_b': matched_school['school_id'],
            'confidence': best_match[1]
        })


In [9]:
# Step 10: Convert results to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv('school_mapping_results.csv', index=False)


In [10]:
school_mapping_results = pd.read_csv('school_mapping_results.csv')

In [11]:
school_mapping_results

,school_id_a,school_id_b,confidence
0,1,12425,50.980392
1,2,13833,53.608247
2,3,12763,62.500000
3,4,4391,48.936170
4,5,434,57.534247
...,...,...,...
20344,29831,34332,43.298969
20345,29832,71662,53.061224
20346,29833,2174,57.142857
20347,29834,36059,55.000000
